# Callbacks

<div class="alert alert-block alert-info">
<b>WARNING:</b> This is a new, experimental feature, and the API may change in future.
</div>

Callbacks provide hooks for users to interact with the different parts of the PyBaMM pipeline, for example to log, save, or visualize outputs of intermediate functions. 

A list of available callbacks can be found in the [API docs](https://pybamm.readthedocs.io/en/latest/source/callbacks.html). Any number of callbacks can be provided as a list, and they will each be called in turn in the order provided.

The base class [`pybamm.callbacks.Callback`](https://pybamm.readthedocs.io/en/latest/source/citations.html#pybamm.callbacks.Callback) documents the available callback methods, at which point in the pipeline they are called, and what arguments are passed to them.

In [1]:
%pip install pybamm -q
import pybamm

model = pybamm.lithium_ion.DFN()
experiment = pybamm.Experiment([
        (
            "Discharge at C/5 for 10 hours or until 3.3 V",
            "Charge at 1 A until 4.1 V",
            "Hold at 4.1 V until 10 mA",
        ),
    ]
    * 3
)
sim = pybamm.Simulation(model, experiment=experiment)

## Logging callback

The `LoggingCallback` can be used to log the progress of the simulation. The default is to print to stdout (this callback is automatically created if no other `LoggingCallback` exists)

In [2]:
pybamm.set_logging_level("NOTICE")
sim.solve();

2021-12-30 19:36:56.258 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/3 (28.051 ms elapsed) --------------------
2021-12-30 19:36:56.259 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:36:57.349 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:36:57.699 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:36:58.161 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/3 (1.931 s elapsed) --------------------
2021-12-30 19:36:58.162 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:36:59.405 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:36:59.660 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:36:59.866 - [NOTICE] callbacks.on_cycle_star

or to a separate file

In [3]:
callback = pybamm.callbacks.LoggingCallback("output.log")
sim.solve(callbacks=callback);

# Read the file that has been written, which was saved to callback.logfile
with open(callback.logfile, "r") as f:
    print(f.read())
    
# Remove the log file
import os
os.remove(callback.logfile)

2021-12-30 19:37:01.578 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/3 (27.159 ms elapsed) --------------------
2021-12-30 19:37:01.578 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:37:02.878 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:37:03.134 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:37:03.542 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/3 (1.992 s elapsed) --------------------
2021-12-30 19:37:03.543 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:37:04.788 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:37:05.041 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:37:05.243 - [NOTICE] callbacks.on_cycle_star

## Custom callbacks

Custom callbacks should subclass the class `pybamm.callbacks.Callback` class, and implement a subset of the callback methods `on_<event>`, which all take as input the dictionary `logs`.

In [4]:
class CustomCallback(pybamm.callbacks.Callback):
    def on_experiment_end(self, logs):
        print(f"We are at the end of the simulation of the simulation. Logs are {logs}")

In [5]:
# Note the default `LoggingCallback` is also called
sim.solve(callbacks=CustomCallback());

2021-12-30 19:37:06.950 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/3 (29.087 ms elapsed) --------------------
2021-12-30 19:37:06.951 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:37:08.408 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:37:08.727 - [NOTICE] callbacks.on_step_start(180): Cycle 1/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:37:09.129 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/3 (2.208 s elapsed) --------------------
2021-12-30 19:37:09.130 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 1/3: Discharge at C/5 for 10 hours or until 3.3 V
2021-12-30 19:37:10.604 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 2/3: Charge at 1 A until 4.1 V
2021-12-30 19:37:10.900 - [NOTICE] callbacks.on_step_start(180): Cycle 2/3, step 3/3: Hold at 4.1 V until 10 mA
2021-12-30 19:37:11.121 - [NOTICE] callbacks.on_cycle_star

We are at the end of the simulation of the simulation. Logs are {'stopping conditions': {'voltage': None, 'capacity': None}, 'cycle number': (3, 3), 'elapsed time': pybamm.TimerTime(4.199561042000001), 'step number': (3, 3), 'step operating conditions': 'Hold at 4.1 V until 10 mA', 'termination': 'event: Current cut-off (negative) [A] [experiment]', 'summary variables': {'Minimum measured discharge capacity [A.h]': -0.16806782223941116, 'Maximum measured discharge capacity [A.h]': 0.6889979156579616, 'Measured capacity [A.h]': 0.8570657378973728, 'Minimum voltage [V]': 3.300010000000005, 'Maximum voltage [V]': 4.100000000000016, 'Positive electrode capacity [A.h]': 1.9464430360887066, 'Change in positive electrode capacity [A.h]': 0.0, 'Loss of active material in positive electrode [%]': 1.1102230246251565e-14, 'Change in loss of active material in positive electrode [%]': 0.0, 'Loss of lithium inventory [%]': -2.220446049250313e-14, 'Change in loss of lithium inventory [%]': 0.0, 'Los